# PanelBox HTML Reports - Complete Guide

**Complete tutorial on generating professional HTML reports with PanelBox visualization system.**

This notebook demonstrates:
1. Creating residual diagnostic charts
2. Model comparison charts  
3. Exporting charts to HTML
4. Multi-format exports
5. Custom themes

---

## 1. Setup and Data Preparation

First, let's import required libraries and prepare sample data.

In [1]:
import numpy as np
import pandas as pd
import panelbox as pb

# Import visualization modules
from panelbox.visualization import (
    create_residual_diagnostics,
    create_comparison_charts,
    ChartFactory
)

print("✅ Libraries imported successfully")
print(f"PanelBox version: {pb.__version__}")

✅ Libraries imported successfully
PanelBox version: 0.8.0


In [2]:
# Load sample dataset (Grunfeld investment data)
data = pb.load_grunfeld()

print(f"Dataset shape: {data.shape}")
print(f"\nFirst few rows:")
data.head()

Dataset shape: (200, 5)

First few rows:


,firm,year,invest,value,capital
0,1,1935,317.6,3078.5,2.8
1,1,1936,391.8,4661.7,52.6
2,1,1937,410.6,5387.1,156.9
3,1,1938,257.7,2792.2,209.2
4,1,1939,330.8,4313.2,203.4


---

## 2. Fit Panel Data Models

Fit different model specifications for comparison.

In [3]:
# 1. Fixed Effects model
fe_model = pb.FixedEffects(
    formula="invest ~ value + capital",
    data=data,
    entity_col="firm",
    time_col="year"
)

fe_results = fe_model.fit(cov_type='robust')

print("✅ Fixed Effects model fitted")
print(f"R-squared: {fe_results.rsquared:.4f}")
print(f"Number of observations: {fe_results.nobs}")

✅ Fixed Effects model fitted
R-squared: 0.7899
Number of observations: 200


In [4]:
# 2. Pooled OLS
pooled = pb.PooledOLS(
    formula="invest ~ value + capital",
    data=data,
    entity_col="firm",
    time_col="year"
)
pooled_results = pooled.fit()

# 3. Random Effects
re = pb.RandomEffects(
    formula="invest ~ value + capital",
    data=data,
    entity_col="firm",
    time_col="year"
)
re_results = re.fit()

print("✅ Three models fitted:")
print(f"   1. Pooled OLS: R² = {pooled_results.rsquared:.4f}")
print(f"   2. Fixed Effects: R² = {fe_results.rsquared:.4f}")
print(f"   3. Random Effects: R² = {re_results.rsquared:.4f}")

✅ Three models fitted:
   1. Pooled OLS: R² = 0.8090
   2. Fixed Effects: R² = 0.7899
   3. Random Effects: R² = 0.8003


---

## 3. Residual Diagnostics Charts

Create comprehensive residual diagnostics visualizations.

In [5]:
# Create all residual diagnostic charts
residual_charts = create_residual_diagnostics(
    fe_results,
    theme='professional'
)

print(f"✅ Created {len(residual_charts)} residual diagnostic charts")
print("\nAvailable charts:")
for name in residual_charts.keys():
    print(f"   • {name}")

✅ Created 6 residual diagnostic charts

Available charts:
   • qq_plot
   • residual_vs_fitted
   • scale_location
   • residual_vs_leverage
   • residual_timeseries
   • residual_distribution


In [6]:
# Export residual charts to HTML
import os
os.makedirs('reports/residual_diagnostics', exist_ok=True)

for chart_name, chart in residual_charts.items():
    output_path = f'reports/residual_diagnostics/{chart_name}.html'
    # Get HTML string and save to file
    html_content = chart.to_html()
    with open(output_path, 'w') as f:
        f.write(html_content)
    print(f"✅ Exported: {output_path}")

print("\n📁 All residual diagnostic charts saved to: reports/residual_diagnostics/")

✅ Exported: reports/residual_diagnostics/qq_plot.html
✅ Exported: reports/residual_diagnostics/residual_vs_fitted.html
✅ Exported: reports/residual_diagnostics/scale_location.html
✅ Exported: reports/residual_diagnostics/residual_vs_leverage.html
✅ Exported: reports/residual_diagnostics/residual_timeseries.html
✅ Exported: reports/residual_diagnostics/residual_distribution.html

📁 All residual diagnostic charts saved to: reports/residual_diagnostics/


---

## 4. Model Comparison Charts

Compare multiple model specifications visually.

In [7]:
# Create model comparison charts
comparison_charts = create_comparison_charts(
    results_list=[pooled_results, fe_results, re_results],
    names=["Pooled OLS", "Fixed Effects", "Random Effects"],
    theme='professional'
)

print(f"✅ Created {len(comparison_charts)} comparison charts")
print("\nAvailable charts:")
for name in comparison_charts.keys():
    print(f"   • {name}")

✅ Created 3 comparison charts

Available charts:
   • coefficients
   • fit_comparison
   • ic_comparison


In [8]:
# Export comparison charts to HTML
os.makedirs('reports/model_comparison', exist_ok=True)

for chart_name, chart in comparison_charts.items():
    output_path = f'reports/model_comparison/{chart_name}.html'
    html_content = chart.to_html()
    with open(output_path, 'w') as f:
        f.write(html_content)
    print(f"✅ Exported: {output_path}")

print("\n📁 All comparison charts saved to: reports/model_comparison/")

✅ Exported: reports/model_comparison/coefficients.html


✅ Exported: reports/model_comparison/fit_comparison.html
✅ Exported: reports/model_comparison/ic_comparison.html

📁 All comparison charts saved to: reports/model_comparison/


---

## 5. Individual Chart Creation

Create specific charts using ChartFactory.

In [9]:
# Create individual charts
from panelbox.visualization import ChartFactory

# Correlation heatmap
corr_data = data[['invest', 'value', 'capital']].corr()
corr_chart = ChartFactory.create(
    'correlation_heatmap',
    {'correlation_matrix': corr_data},
    theme='professional'
)

print("✅ Created correlation heatmap")

✅ Created correlation heatmap


In [10]:
# Export individual chart
os.makedirs('reports/individual_charts', exist_ok=True)

html_content = corr_chart.to_html()
with open('reports/individual_charts/correlation_heatmap.html', 'w') as f:
    f.write(html_content)

print("✅ Exported: reports/individual_charts/correlation_heatmap.html")

✅ Exported: reports/individual_charts/correlation_heatmap.html


---

## 6. Multi-Format Export

Export charts in multiple formats (HTML, JSON).

In [11]:
# Select key charts for multi-format export
key_charts = {
    'residual_qq': residual_charts['qq_plot'],
    'residual_vs_fitted': residual_charts['residual_vs_fitted'],
    'coefficients': comparison_charts['coefficients']
}

# Export to multiple formats
os.makedirs('reports/multiformat', exist_ok=True)

for chart_name, chart in key_charts.items():
    # HTML
    html_content = chart.to_html()
    with open(f'reports/multiformat/{chart_name}.html', 'w') as f:
        f.write(html_content)
    
    # JSON
    json_content = chart.to_json()
    with open(f'reports/multiformat/{chart_name}.json', 'w') as f:
        f.write(json_content)
    
    print(f"✅ Exported {chart_name} in multiple formats")

print("\n📁 Multi-format charts saved to: reports/multiformat/")
print("\nFormats available:")
print("   • HTML - Interactive web version")
print("   • JSON - Plotly specification")

✅ Exported residual_qq in multiple formats
✅ Exported residual_vs_fitted in multiple formats
✅ Exported coefficients in multiple formats

📁 Multi-format charts saved to: reports/multiformat/

Formats available:
   • HTML - Interactive web version
   • JSON - Plotly specification


---

## 7. Custom Themes

Apply different themes to visualizations.

In [12]:
# Try different themes
themes = ['professional', 'academic', 'presentation']

for theme_name in themes:
    # Create charts with specific theme
    themed_charts = create_residual_diagnostics(
        fe_results,
        charts=['qq_plot'],  # Just one chart for demo
        theme=theme_name
    )
    
    # Export
    os.makedirs(f'reports/themes/{theme_name}', exist_ok=True)
    html_content = themed_charts['qq_plot'].to_html()
    with open(f'reports/themes/{theme_name}/qq_plot.html', 'w') as f:
        f.write(html_content)
    
    print(f"✅ Exported with '{theme_name}' theme")

print("\n🎨 Compare themes at:")
for theme in themes:
    print(f"   • reports/themes/{theme}/qq_plot.html")

✅ Exported with 'professional' theme


✅ Exported with 'academic' theme
✅ Exported with 'presentation' theme

🎨 Compare themes at:
   • reports/themes/professional/qq_plot.html
   • reports/themes/academic/qq_plot.html
   • reports/themes/presentation/qq_plot.html


---

## 8. Report Organization

Create a master index for all reports.

In [13]:
# Create master HTML index
master_html = '''<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>PanelBox Analysis Reports</title>
    <style>
        body {{
            font-family: system-ui, -apple-system, sans-serif;
            max-width: 1200px;
            margin: 0 auto;
            padding: 20px;
            background: #f5f5f5;
        }}
        h1 {{ color: #1e3a8a; border-bottom: 3px solid #2563eb; padding-bottom: 10px; }}
        h2 {{ color: #1e40af; margin-top: 40px; }}
        .section {{
            background: white;
            padding: 20px;
            margin: 20px 0;
            border-radius: 8px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        }}
        .report-link {{
            display: inline-block;
            padding: 10px 20px;
            margin: 10px 10px 10px 0;
            background: #2563eb;
            color: white;
            text-decoration: none;
            border-radius: 6px;
        }}
        .report-link:hover {{ background: #1d4ed8; }}
    </style>
</head>
<body>
    <h1>📊 PanelBox Analysis Reports</h1>
    
    <div class="section">
        <h2>Model Information</h2>
        <p><strong>Formula:</strong> invest ~ value + capital</p>
        <p><strong>Dataset:</strong> Grunfeld Investment Data</p>
        <p><strong>Observations:</strong> {nobs}</p>
        <p><strong>Fixed Effects R²:</strong> {rsquared:.4f}</p>
    </div>
    
    <div class="section">
        <h2>📋 Available Reports</h2>
        
        <h3>1. Residual Diagnostics</h3>
        <a href="residual_diagnostics/qq_plot.html" class="report-link">Q-Q Plot</a>
        <a href="residual_diagnostics/residual_vs_fitted.html" class="report-link">Residual vs Fitted</a>
        <a href="residual_diagnostics/scale_location.html" class="report-link">Scale-Location</a>
        <a href="residual_diagnostics/residual_vs_leverage.html" class="report-link">Residual vs Leverage</a>
        
        <h3>2. Model Comparison</h3>
        <a href="model_comparison/coefficients.html" class="report-link">Coefficient Comparison</a>
        <a href="model_comparison/model_fit.html" class="report-link">Model Fit Statistics</a>
        
        <h3>3. Individual Charts</h3>
        <a href="individual_charts/correlation_heatmap.html" class="report-link">Correlation Heatmap</a>
    </div>
    
    <div class="section">
        <p style="text-align: center; color: #666;">
            Generated with <strong>PanelBox v{version}</strong>
        </p>
    </div>
</body>
</html>
'''.format(
    nobs=fe_results.nobs,
    rsquared=fe_results.rsquared,
    version=pb.__version__
)

# Save master index
with open('reports/index.html', 'w') as f:
    f.write(master_html)

print("✅ Master index created: reports/index.html")
print("\n🎯 Open this file in your browser to access all reports!")

✅ Master index created: reports/index.html

🎯 Open this file in your browser to access all reports!


---

## Summary

### ✅ What We Created

1. **Residual Diagnostics** - 6 diagnostic charts
2. **Model Comparison** - Coefficient and fit comparisons
3. **Individual Charts** - Correlation heatmap
4. **Multi-Format Exports** - HTML and JSON formats
5. **Custom Themes** - Professional, academic, presentation
6. **Master Index** - Central navigation for all reports

### 📁 Generated Files

```
reports/
├── index.html                      # Master dashboard
├── residual_diagnostics/           # Diagnostic charts
│   ├── qq_plot.html
│   ├── residual_vs_fitted.html
│   ├── scale_location.html
│   ├── residual_vs_leverage.html
│   ├── residual_timeseries.html
│   └── residual_distribution.html
├── model_comparison/               # Model comparison
│   ├── coefficients.html
│   └── model_fit.html
├── individual_charts/              # Custom charts
│   └── correlation_heatmap.html
├── multiformat/                    # Multi-format exports
│   ├── *.html
│   └── *.json
└── themes/                         # Theme demonstrations
    ├── professional/
    ├── academic/
    └── presentation/
```

### 🚀 Next Steps

1. Open `reports/index.html` in your browser
2. Explore the interactive charts
3. Customize themes for your needs
4. Share reports with collaborators

### 📚 Additional Resources

- **Chart Gallery**: `examples/CHART_GALLERY.md`
- **Chart Selection Guide**: `examples/README_CHART_SELECTION.md`
- **Custom Themes Tutorial**: `examples/custom_themes_tutorial.md`

---

**Happy Reporting! 📊✨**